In [1]:
import csv
from itertools import count
from matplotlib.pyplot import polar
import pandas as pd
from email import header
import re
import nltk
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
nltk.download('omw-1.4')
import gensim
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words('english')
newStpWord = [',','–','!','"','#','$','%','&',')','*','+','&',"i'm","I'm","I've",'you','me',"i've",'0','1','2','3','4','5','6','7','8','9','?','and','or','however','course.','course',"much",'more']
nltk_stopwords.extend(newStpWord)
from nltk.tokenize import word_tokenize
import gensim.corpora as corpora
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from pprint import pprint
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
#import pyLDAvis.gensim
from string import punctuation
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
nltk.download('wordnet')
from nltk.corpus import wordnet 
from wordcloud import WordCloud, STOPWORDS
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\Danny\anaconda3\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nl

True

In [2]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
pos = []
neg = []
sent_ls = [] #存欲分類正負情感之字串
path = 'E:\python\Sentiment&LDA\Import_Data_complete_final_sentenized.csv'
f = open(path, 'r',encoding="utf-8")
rows = csv.reader(f, delimiter=',')
for row in rows:
    sent_ls.append(row[2])

res = [idx for idx in sent_ls if not re.findall("[^\u0000-\u05C0\u2100-\u214F]+", idx)] #res = 20114筆
clean_data = []
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemma(text):
    
    lst_text = nltk.word_tokenize(text)
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    lemmatized_output = ' '.join([lem.lemmatize(w) for w in lst_text])
    #lst_text = [lem.lemmatize(word) for word in lst_text]
    
    #if lst_text == text:
    #    lst_text = [lem.lemmatize(word, 'n') for word in text]
    #text = " ".join(lst_text)
    return lemmatized_output

for i in res:
    p = re.sub(r'[.,"\'-?:!;]', '', i)
    txt_lemma = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(p)]
    text = " ".join(txt_lemma)
    clean_data.append(text) 
sent_analyzer = SentimentIntensityAnalyzer()

test = pd.DataFrame(clean_data)
test.columns = ["reviews"]

def format_output(output_dict):
    polarity = "neutral"
    if (output_dict['compound']>=0.05):
        polarity = "positive"
    elif(output_dict['compound']<=-0.05):
        polarity = "negative"
    return polarity

def sentiment_calc(text):
     try:
         return sent_analyzer.polarity_scores(text)
     except:
         return None
test['sentiment_polarity'] = test['reviews'].apply(sentiment_calc)
test['pos_or_neg'] = test['sentiment_polarity'].apply(format_output)

for row in range (len(clean_data)):
    if test.loc[row][2]=="positive":
        pos.append(test.loc[row][0])       #positive review array list
    elif test.loc[row][2]=="negative":
        neg.append(test.loc[row][0])       #negative review array list

newStpWord_2 = ['game','Game', "i'm","I'm","I've",       #extend sentiment stopwords fo pos and neg
"i've",'0','1','2','3','4','5','6','7','8','9','?','sorry','course.','dr','chuck','get','excellent','make','let','take','el','la','cod','quite','however','other','others','arduino','give','be','iam','im','ive','one','thank','yes','no','not','do','can','this','it','to','the','course','best','good','bad','great',"'great",",good",",great","'amazing",'``',"`",'nice','java',
'like','love','lot','little','go','really','well','python','easy','me','you','programming','course,',"coursera",'could',"'good","''","'",'would','many','much','also','awesome','excellent']


def senti_word_rem(text):  #remove sentiment words function
    arr = []
    for i in text:
        x = i.split()
        filter = [k for k in x if not k in newStpWord_2] 
        filter = ' '.join(filter)
        arr.append(filter)
    return arr
clean_data_without_SentiWord = senti_word_rem(clean_data)

def gen_words(texts):
    final_pos = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final_pos.append(new)
    return (final_pos)

#N-gram process def
def to_bigram(text, count, thrs):
    bi_list = []
    bigram = gensim.models.Phrases(text, min_count=count, threshold=thrs) # higher threshold fewer phrases. #★★★★★★highest value★★★★★★★
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    for i in range(len(text)):
        bi_list.append(bigram_mod[text[i]])
    return bi_list

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values




data_words_overall = gen_words(clean_data_without_SentiWord)
data_words_pos = to_bigram(gen_words(senti_word_rem(pos)), 5, 100)
data_words_neg = to_bigram(gen_words(senti_word_rem(neg)), 7, 100)



#print("ovreall rev")
print(len(clean_data_without_SentiWord))
print("pos rev:")
print(len(data_words_pos))
print("neg rev:")
print(len(data_words_neg))



<>:5: DeprecationWarning: invalid escape sequence \p
<>:5: DeprecationWarning: invalid escape sequence \p
C:\Users\Danny\AppData\Local\Temp\ipykernel_16424\3137898726.py:5: DeprecationWarning: invalid escape sequence \p
  path = 'E:\python\Sentiment&LDA\Import_Data_complete_final_sentenized.csv'


96845
pos rev:
73313
neg rev:
4873


In [3]:
pos_words = ' '.join([str(i) for i in data_words_pos])
neg_words = ' '.join([str(i) for i in data_words_neg])

In [ ]:
wordcloud_pos = WordCloud().generate(pos_words)
# 繪圖
plt.figure()
print(wordcloud_pos)
print("Positive review WordCloud")
plt.imshow(wordcloud_pos, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wordcloud_neg = WordCloud().generate(neg_words)
# 繪圖
plt.figure()
print("Negative review WordCloud")
plt.imshow(wordcloud_neg, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
test.to_csv("Complete_clean_dataset.csv", encoding='utf-8')

In [ ]:
# # #positive review coherence
# id2word_pos = corpora.Dictionary(data_words_pos)
# corpus_pos = []
# for text in data_words_pos:
#    new = id2word_pos.doc2bow(text)
#    corpus_pos.append(new)
# coherence_list = []
# for i in range(2,15):
#    bi_lda_model_pos = gensim.models.ldamodel.LdaModel(corpus=corpus_pos,
#                                           id2word=id2word_pos,
#                                           num_topics=i,
#                                           random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                           passes=10,
#                                           alpha="auto")
#    coherence_model_lda = CoherenceModel(model=bi_lda_model_pos, texts=data_words_pos, dictionary=id2word_pos , coherence='c_v')
#    coherence_lda = coherence_model_lda.get_coherence()
#    coherence_list.append(coherence_lda)
# print(coherence_list)
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_list)
# plt.xlabel("Num Topics")
# plt.ylabel("Positive Review Bigram Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [ ]:
# # #negative review coherence
# id2word_neg = corpora.Dictionary(data_words_neg)
# corpus_neg = []
# for text in data_words_neg:
#    new = id2word_neg.doc2bow(text)
#    corpus_neg.append(new)
# coherence_list = []
# for i in range(2,15):
#    lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus_neg,
#                                           id2word=id2word_neg,
#                                           num_topics=i,
#                                           random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                          passes=10,
#                                           alpha="auto")
#    coherence_model_lda = CoherenceModel(model=lda_model_neg, texts=data_words_neg, dictionary=id2word_neg , coherence='c_v')
#    coherence_lda = coherence_model_lda.get_coherence()
#    coherence_list.append(coherence_lda)
# print(coherence_list)
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_list)
# plt.xlabel("Num Topics")
# plt.ylabel("Negative Review Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [ ]:
#Overall review coherence
#Overall topic num: 8
#model_list, coherence_values = compute_coherence_values(dictionary=id2word_ov, corpus=corpus_ov, texts=data_words_ov, start=2, limit=15, step=1) 
# Show graph
import matplotlib.pyplot as plt
#limit=15; start=2; step=1;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
#plt.show()

In [ ]:
#Positive review coherence
#Positive topic num: 8
#model_list, coherence_values = compute_coherence_values(dictionary=id2word_pos, corpus=corpus_pos, texts=data_words_pos, start=2, limit=15, step=1) 
# Show graph
#import matplotlib.pyplot as plt
#limit=15; start=2; step=1;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
#plt.show()

In [ ]:
#Negative review coherence
#Negative topic num: 7
#model_list, coherence_values = compute_coherence_values(dictionary=id2word_neg, corpus=corpus_neg, texts=data_words_neg, start=2, limit=15, step=1) 
# Show graph
#import matplotlib.pyplot as plt
#limit=15; start=2; step=1;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
#plt.show()

In [ ]:
# #positive review coherence
# coherence_list = []
# for i in range(2,15):
#    lda_model_pos = gensim.models.ldamodel.LdaModel(corpus=corpus_pos,
#                                           id2word=id2word_pos,
#                                           num_topics=i,
#                                           random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                           passes=10,
#                                           alpha="auto")
#    coherence_model_lda = CoherenceModel(model=lda_model_pos, texts=data_words_pos, dictionary=id2word_pos , coherence='c_v')
#    coherence_lda = coherence_model_lda.get_coherence()
#    coherence_list.append(coherence_lda)
# print(coherence_list)
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_list)
# plt.xlabel("Num Topics")
# plt.ylabel("Positive Review Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [ ]:
# #positive review coherence
# coherence_list = []
# for i in range(2,15):
#    lda_model_pos = gensim.models.ldamodel.LdaModel(corpus=corpus_pos,
#                                           id2word=id2word_pos,
#                                           num_topics=i,
#                                           random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                           passes=10,
#                                           alpha="auto")
#    coherence_model_lda = CoherenceModel(model=lda_model_pos, texts=data_words_pos, dictionary=id2word_pos , coherence='u_mass')
#    coherence_lda = coherence_model_lda.get_coherence()
#    coherence_list.append(coherence_lda)
# print(coherence_list)
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_list)
# plt.xlabel("Num Topics")
# plt.ylabel("Positive Review Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [ ]:
# #negative review coherence
# coherence_list = []
# for i in range(2,15):
#    lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus_neg,
#                                           id2word=id2word_neg,
#                                           num_topics=i,
#                                           random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                          passes=10,
#                                           alpha="auto")
#    coherence_model_lda = CoherenceModel(model=lda_model_neg, texts=data_words_neg, dictionary=id2word_neg , coherence='c_v')
#    coherence_lda = coherence_model_lda.get_coherence()
#    coherence_list.append(coherence_lda)
# print(coherence_list)
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_list)
# plt.xlabel("Num Topics")
# plt.ylabel("Negative Review Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [ ]:
# #negative review coherence
# coherence_list = []
# for i in range(2,15):
#    lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus_neg,
#                                           id2word=id2word_neg,
#                                           num_topics=i,
#                                           random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                          passes=10,
#                                           alpha="auto")
#    coherence_model_lda = CoherenceModel(model=lda_model_neg, texts=data_words_neg, dictionary=id2word_neg , coherence='u_mass')
#    coherence_lda = coherence_model_lda.get_coherence()
#    coherence_list.append(coherence_lda)
# print(coherence_list)
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_list)
# plt.xlabel("Num Topics")
# plt.ylabel("Negative Review Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [ ]:
# def preplexity(num_topic, corpus,dic):
#     ladmodel = gensim.models.ldamodel.LdaModel(corpus = corpus, num_topics = num_topic, id2word = dic,random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                          passes=10,
#                                           alpha="auto")
#     return ladmodel.log_perplexity(corpus)

# #print(preplexity(1,corpus_pos,id2word_pos))
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# y = []
# for i in range(2,15):
#     y.append(preplexity(i,corpus_pos, id2word_pos))
# print(y)
# plt.plot(x, y)
# plt.xlabel("Num Topics")
# plt.ylabel("Positive Review Perplexity score")
# plt.legend(("perplexity_values"), loc='best')
# plt.show()    

In [ ]:
# limit=15; start=2; step=1;
# x = range(start, limit, step)
# y = []
# for i in range(2,15):
#     y.append(preplexity(i,corpus_neg, id2word_neg))
# print(y)
# plt.plot(x, y)
# plt.xlabel("Num Topics")
# plt.ylabel("Negative Review Perplexity score")
# plt.legend(("perplexity_values"), loc='best')
# plt.show()    

In [ ]:
corpus_ov = []
id2word_ov=corpora.Dictionary(data_words_overall)
for text in data_words_overall:
    new = id2word_ov.doc2bow(text)
    corpus_ov.append(new)

In [4]:

#cm = CoherenceModel(model=lda_model_ov, corpus=corpus_ov, coherence='u_mass')
#coherence = cm.get_coherence()
#print('\nOverall Review Cohernece: ',coherence)

#if __name__=='__main__':
#    coherence_model_lda = CoherenceModel(model=lda_model_ov, texts=clean_data_without_SentiWord, dictionary=id2word_ov, coherence='c_v')
#    coherence_lda = coherence_model_lda.get_coherence()
#    print('\nCoherence Score: ', coherence_lda)
#--------------------------positive lda analysis--------------------------
id2word_pos = corpora.Dictionary(data_words_pos)
corpus_pos = []
for text in data_words_pos:
    new = id2word_pos.doc2bow(text)
    corpus_pos.append(new)
lda_model_pos = gensim.models.ldamodel.LdaModel(corpus=corpus_pos,
                                           id2word=id2word_pos,
                                           num_topics=12,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")
pprint(lda_model_pos.print_topics(12))     
p_doc_term_matrix_pos = [id2word_pos.doc2bow(doc) for doc in data_words_pos]

# Compute Perplexity
print('\nPositive Review Perplexity: ', lda_model_pos.log_perplexity(p_doc_term_matrix_pos))   #Positive Review Perplexity:  -7.7604093115426585
# a measure of how good the model is. lower the better.


#--------------------------negative lda analysis--------------------------
id2word_neg = corpora.Dictionary(data_words_neg)
corpus_neg = []
for text in data_words_neg:
    new = id2word_neg.doc2bow(text)
    corpus_neg.append(new)
lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus_neg,
                                           id2word=id2word_neg,
                                           num_topics=13,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")
pprint(lda_model_neg.print_topics(13))   
p_doc_term_matrix_neg = [id2word_neg.doc2bow(doc) for doc in data_words_neg]
# Compute Perplexity
print('\nNegative Review Perplexity: ', lda_model_neg.log_perplexity(p_doc_term_matrix_neg))   #Negative Review Perplexity:  -9.519325542067824
# a measure of how good the model is. lower the better.


[(0,
  '0.096*"understand" + 0.084*"interest" + 0.078*"beginner" + 0.067*"concept" '
  '+ 0.057*"way" + 0.052*"helpful" + 0.039*"clear" + 0.032*"example" + '
  '0.024*"practical" + 0.021*"simple"'),
 (1,
  '0.125*"challenge" + 0.106*"amaze" + 0.053*"cover" + 0.045*"last" + '
  '0.033*"sir" + 0.030*"important" + 0.025*"apply" + 0.024*"real" + '
  '0.024*"finish" + 0.021*"difficult"'),
 (2,
  '0.112*"experience" + 0.092*"week" + 0.060*"student" + 0.049*"exercise" + '
  '0.039*"job" + 0.032*"specialization" + 0.026*"found" + 0.025*"every" + '
  '0.024*"implement" + 0.023*"worth"'),
 (3,
  '0.120*"fun" + 0.103*"teach" + 0.065*"new" + 0.049*"even" + 0.031*"hard" + '
  '0.030*"build" + 0.029*"theory" + 0.028*"create" + 0.026*"fantastic" + '
  '0.025*"happy"'),
 (4,
  '0.193*"program" + 0.076*"introduction" + 0.073*"parallel" + 0.060*"start" + '
  '0.049*"thing" + 0.035*"think" + 0.027*"language" + 0.027*"overall" + '
  '0.027*"bit" + 0.025*"structure"'),
 (5,
  '0.121*"thanks" + 0.096*"expla

In [ ]:
# wordcloud = WordCloud().generate(data_words_pos)

# # 繪圖
# plt.figure()
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis("off")
# plt.show()

In [ ]:
#Positive Vis
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_pos, corpus_pos, id2word_pos, mds="tsne")
vis

In [ ]:
#Negative Vis
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_neg, corpus_neg, id2word_neg, mds="tsne")
vis

In [ ]:
# #N-gram process Positive
# # Build the positive bigram and trigram models
# bigram = gensim.models.Phrases(data_words_pos, min_count=5, threshold=100) # higher threshold fewer phrases. #★★★★★★highest value★★★★★★★
# trigram = gensim.models.Phrases(bigram[data_words_pos], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# bi_data_words_pos = []
# # See trigram example
# for i in range(len(data_words_pos)):
#     bi_data_words_pos.append(bigram_mod[data_words_pos[i]])
# print(bi_data_words_pos)
# #print(trigram_mod[bigram_mod[data_words_pos[0]]])

In [ ]:
# #N-gram process Negative
# # Build the positive bigram and trigram models
# bigram = gensim.models.Phrases(data_words_neg, min_count=7, threshold=100) # higher threshold fewer phrases. #★★★★★★highest value★★★★★★★
# trigram = gensim.models.Phrases(bigram[data_words_neg], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# bi_data_words_neg = []
# # See trigram example
# for i in range(len(data_words_neg)):
#     bi_data_words_neg.append(bigram_mod[data_words_neg[i]])
# print(bi_data_words_neg)
# #print(trigram_mod[bigram_mod[data_words_pos[0]]])